# Notebook for retrieving final destination phishing domains from urlscan.io 

## Requirements
* Python Libraries
  * Pandas
  * Requests
* urlscan.io API Key

In [ ]:
# Libs
from requests import get
from getpass import getpass
from urllib.parse import quote
from json import loads
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## Get urlscan.io data

### REGEXs
We have 2 REGEX queries that will find all the relevant final destination (FD) phishing domains with high accuracy. There are a few FPs that do come up however, we can filter those results out in later cell blocks. While we could combine these queries together with a optional capture group, i.e. `(\-)?`, we are rate limited by 10000 results for each query by urlscan.io. 

#### REGEX 1: All FD domains with `-`
**Base REGEX**

```
^(http)(s)?:\/\/([a-z0-9]+\.)?[a-z0-9]{1,}\-[a-z0-9]{1,}\.(shop|online|xyz|club|com|top|si|org|net|live)\/main\/$
```

**urlscan.io Query**

```
page.url.keyword:/(http)(s)?:\/\/([a-z0-9]+\.)?[a-z0-9]{1,}\-[a-z0-9]{1,}\.(shop|online|xyz|club|com|top|si|org|net|live)\/main\//
```

#### REGEX 2: All FD domains without `-`
**Base REGEX**

```
^(http)(s)?:\/\/([a-z0-9]+\.)?([a-z]|[0-9])+\.(online|xyz|com|top|club|com.au|org.ge|co.ug|co.mz|one|org|net|live)\/main\/$
```

**urlscan.io Query**
```
page.url.keyword:/(http)(s)?:\/\/([a-z0-9]+\.)?([a-z]|[0-9])+\.(online|xyz|com|top|club|com.au|org.ge|co.ug|co.mz|one|org|net|live)\/main\//
```


### Getting Data
Using python requests can be slow, so sometimes using curl is better for grabbing a ton of results if you are in a time crunch.
The first code block uses Python's requests library while the next code block loads from a file.

#### Python Requests

**Enter API Key**

In [ ]:
api_key = getpass()

In [ ]:
# Set max results here!
max_results = 100
base_url = f'https://urlscan.io/api/v1/search/?size={max_results}&q='
regex_one_query = quote(
    'page.url.keyword:/(http)(s)?:\/\/([a-z0-9]+\.)?[a-z0-9]{1,}\-[a-z0-9]{1,}\.(shop|online|xyz|club|com|top|si|org|net|live)\/main\//'
)
regex_two_query = quote(
    'page.url.keyword:/(http)(s)?:\/\/([a-z0-9]+\.)?([a-z]|[0-9])+\.(online|xyz|com|top|club|com.au|org.ge|co.ug|co.mz|one|org|net|live)\/main\//'
)
headers = {'api-key': api_key}

In [ ]:
regex_one_results = get(f'{base_url}{regex_one_query}', headers=headers)

In [ ]:
regex_two_results = get(f'{base_url}{regex_two_query}', headers=headers)

#### Parse results from json file

In [ ]:
# Parse results from json file
regex_one_file = ''
regex_two_file = ''

with open(regex_one_file, 'r') as file:
    regex_one_results = loads(file.read())

with open(regex_two_file, 'r') as file:
    regex_two_results = loads(file.read())

In [ ]:
print(f'REGEX one results: {regex_one_results["total"]}')
print(f'REGEX two results: {regex_two_results["total"]}')

# Combine data
regex_results = regex_one_results['results'] + regex_two_results['results']
total_results_prefilter = len(regex_results)
print(f'Total results: {total_results_prefilter}')

## Filter False Positives
* Where path is `/Main/` and not `/main/`
* Where task url is `/main/`. While this could remove some TPs, the FDs will likely pop up in other results.
* Where task url's path is root
* Remove google redirects
* Where task url path does at least 1 `?` or `@` or `==` or `#` or `&`
* Where domains are on the alexa 1 million

In [ ]:
# Alexa 1 Million (domain line by line)
alexa_file = ''

In [ ]:
with open(alexa_file, 'r') as alexa:
    alexa = alexa.read().splitlines()

In [ ]:
filtered_regex_results = regex_results.copy()
for i in regex_results:
    if '.'.join(i['page']['domain'].split('.')[-2:]) in alexa or i['page']['url'][-6:] == '/Main/' or i['task']['url'][-6:] == '/main/' or '/goo.gl/' in i['task']['url'] or 'google.com/' in i['task']['url'] or ('&' not in i['task']['url'][8:] and '?' not in i['task']['url'] and '@' not in i['task']['url'] and '==' not in i['task']['url'] and '#' not in i['task']['url']):
        x = 0
        while x < len(filtered_regex_results):
            if filtered_regex_results[x]['_id'] == i['_id']:
                del filtered_regex_results[x]
                break
            x += 1
        continue

In [ ]:
print(f'Results removed: {total_results_prefilter - len(filtered_regex_results)}')
print(f'New total: {len(filtered_regex_results)}')

## Output Final Destination Phishing Domains

**Enter output file**

In [ ]:
output_file = 'phish_domains.txt'

In [ ]:
# Dedup domains
domains = set()
for i in filtered_regex_results:
    domains.add(i['page']['domain'])

# Write to file
with open(output_file, 'w+') as file:
    for domain in domains:
        file.write(f"{domain}\n")

**Optional: output phish domains with datetime**

In [ ]:
output_file = 'phish_domains_with_datetime.csv'

In [ ]:
tasks_and_page_urls = []
for i in filtered_regex_results:
    tasks_and_page_urls.append([i['task']['time'], i['page']['domain']])
df = pd.DataFrame(tasks_and_page_urls, columns=['Task Time', 'Domain'])

In [ ]:
with open(output_file, 'w+') as file:
    file.write(df.to_csv(index=False))